# One-Dimensional linear Aquifer Thermal Energy Storage (ATES) with the GroundWater Engergy (GWE) package

In this notebook, we will learn how to:
1. Setup a MODFLOW 6 model for one-dimensional flow and energy transport.
2. Simulate injection of water that is warmer or colder than the background groundwater. 
3. Check the energy balance and assess numerical dispersion.
4. Simulate a full cycle of injection and recovery.
5. Compute the thermal recovery efficiency.

In [ ]:
# import the necessary packages
import numpy as np # the numpy package
import matplotlib.pyplot as plt # the plotting part of matplotlib
plt.rcParams['figure.figsize'] = (8, 2.5) # set default figure size
import flopy as fp  # import flopy and call it fp
from ipywidgets import interact, FloatSlider, Layout # import some widget functions
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # don't show DeprecationWarning-s in this notebook

## Description of the flow problem
Consider one-dimensional flow in a semi-infinite confined aquifer. The aquifer extends from $x=0$ to $x=L$ in the $x$ direction, where $L$ is chosen far enough away not to effect the solution.  Water is injected at the left side at a rate $U$ such that the average velocity in the aquifer is 1 m/d. The head is fixed on the right side to $h_R$. Flow is considered to be at steady state instantaneously. 
The initial temperature is $T_\text{init}$ everywhere. Injection of warmer water with temperature $T_\text{inj}$ starts at $t=0$ and lasts for $t_\text{in}$ days. Buoyancy and viscosity changes are not considered as the temperature differences are small.

### Parameters

In [ ]:
# domain size and boundary conditions
L = 150 # length of domain, m
hR = 0 # head at right side of domain

# aquifer parameters
k = 35 # hydraulic conductivity, m/d
H = 20 # aquifer thickness, m
theta = 0.3 # porosity, -

# flow parameters
U = 6 # total inflow, m^3/m/d

# energy parameters
rhow = 1000 # density of water, kg/m^3
rhos = 2640 # density of solids, kg/m^3
kappaw = 0.58 * 86400 # thermal conductivity water, J/(d m C)
kappas = 3 * 86400 # thermal conductivity solids, J/(d m C)
cpw = 4180 # specific heat capacity water (mass based), J/(kg C)
cps = 710 # specific heat capacity solids (mass based), J/(kg C)

# transport
alphaL = 0.1 # longitudinal dispersivity in horizontal direction, m
alphaT = alphaL / 10 # transverse dispersivity is 10 times smaller than longitudinal, m

# temperature
Tinit = 12 # initial temperature, C
Tinj = 17 # temperature injected water, C

# space discretization
delr = 1 # length of cell along row (in x-direction), m
delc = 1 # width of cells normal to plane of flow (in y-direction), m
nlay = 1 # number of layers
nrow = 1 # number of rows
ncol = round(L / delr) # number of columns, integer
z = np.linspace(0, -H, nlay + 1) # top and bottom(s) of layers
xg = np.cumsum(delr * np.ones(ncol)) - delr / 2 # x-values centers of grid cells, m

# time and time discretization
tin = 180 # injection time, d
delt = 1 # time step, d
nstepin = round(tin / delt) # computed number of steps during injection, integer

# model name and workspace
modelname = 'gwe1d' # name of model
gwfname = modelname + 'f' # name of flow model
gwename = modelname + 'e' # name of energy model
modelws = './' + modelname # model workspace to be used (where MODFLOW will store all the files)

In [ ]:
# compute average velocity
vavg = U / H / theta
print(f'average velocity: {vavg} m/d')

In MODFLOW6 the first step is to create a so-called `Simulation`. A simulation may contain one or more models. Here, the simulation contains a groundwater flow model and a groundwater energy model. These models may use different solvers, but share the same time discretization.

The general process of generating a combined groundwater flow and energy simulation in MODFLOW6 is:
* Define the model name and workspace
* Create simulation and call it `sim`
  * Add time discretization to `sim`
  * Add groundwater flow model (called `gwf`) to `sim`
    * Add iterative model solver to `gwf` and register solver
    * Add features to groundwater model `gwf` (multiple steps)
  * Add energy model (called `gwe`) to `sim`
    * Add iterative model solver to `gwe` and register solver
    * Add features to energy model `gwe` (multiple steps)
  * Define the interaction between the groundwater flow model and the energy model
* Write MODFLOW input files for `sim`
* Solve simulation `sim`
* Read MODFLOW output for `sim`
* Visualize output

## Create Simulation
Create a simulation and store it in the variable `sim`. The meaning of the keywords is explained in the comments at the end of each line (unless it is self-explanatory).

The time discretization is defined as part of the simulation. The time units are defined as days, but this is used only to control the text in the output files. The user is responsible for making sure that the length and time units of all model variables are consistent. Here, we use meters and days.

Each simulation is divided into stress periods. 
All stresses are constant during a stress period. 
For each stress period,  the length of the period, the number of steps during the period, and a timestep multiplier must be specified.
The `perioddata` is a list with a list of `[period length, number of steps, timestep multiplier]` for each stress period.  

In [ ]:
# simulation
sim = fp.mf6.MFSimulation(sim_name=modelname, # name of simulation
                          version='mf6', # version of MODFLOW
                          exe_name='mf6', # path to MODFLOW executable
                          sim_ws=modelws, # path to workspace where all files are stored
                         )

# time discretization
tdis = fp.mf6.ModflowTdis(simulation=sim, # add to the simulation called sim (defined above)
                          time_units="DAYS", 
                          nper=1, # number of stress periods 
                          perioddata=[[tin, nstepin, 1]], # [period length, number of steps, timestep multiplier] for each stress period
                         )

## Create groundwater flow model (`gwf`)
A groundwater flow model is added to the simulation. The groundwater flow model is stored in the variable `gwf`.
The following features are added to the groundwater flow model `gwf`:
* Iterative model solver \
  The iterative model solver is defined and added to the simulation (not the groundwater model) and must include a separate file name. After creation, the iterative model solver needs to be registered with the simulation (this is a somewhat strange step that can hopefully be done automatically by `flopy` in the future). 
* Spatial discretization \
  All function arguments are self-explanatory and previously defined. 
* Aquifer properties \
  Specify the hydraulic conductivity of the layers (a constant value here, but it may be different for each cell).
* Initial conditions \
  Initial heads. Initial conditions must be specified for all simulations, even for steady simulations (where they serve as starting values for the iterative solution).
* Inflow on the left side of the model \
  Inflow is simulated by specifying a well in the first cell (column 0) by specifying the location and discharge (positive for injecting water, negative for extracting water) and the temperature of the injected water (the temperature is an auxiliary variable and it must be specified explicitly). Well discharges are constant for a stress period but may change from one stress period to another. The `stress_period_data` is a dictionary with all the wells for each stress period (we start out with only one well), starting with period 0. A nested list is provided for each stress period with a list for each well containing a tuple with (layer, row, column) of the cell where the well is located, the discharge, and the temperature of the injected water. 
```
{stress_period number: [[(layer, row, column), discharge, conc]],
 stress_period number: [[(layer, row, column), discharge, conc]], ... }
```
* Specified head on the right side of the model \
  Specified head cells are entered in a similar manner as wells by specifying the stress period data, but now the discharge is replaced by the specified_head.
* Output control \
  The output control package determines which output is saved and how frequent. The `saverecord` keyword has many options to save only part of the model output (useful for large models), for example heads or water budget terms. For this small model, we save `ALL` values.

In [ ]:
# groundwater flow model
gwf = fp.mf6.ModflowGwf(simulation=sim, # add to simulation called sim
                        modelname=gwfname, # name of gwf model
                        save_flows=True, # make sure all flows are stored in binary output file
                       )

# iterative model solver
gwf_ims  = fp.mf6.ModflowIms(simulation=sim, # add to simulation called sim
                             filename=gwf.name + '.ims', # file name to store ims
                             linear_acceleration="BICGSTAB", # use BIConjuGantGradientSTABalized method
                            )                                                                                                
# register solver
sim.register_ims_package(solution_file=gwf_ims, # name of iterative model solver instance
                         model_list=[gwf.name], # list with name of groundwater flow model
                        )   

# discretization
gwf_dis = fp.mf6.ModflowGwfdis(model=gwf, # add to groundwater flow model called gwf
                               nlay=nlay, 
                               nrow=nrow, 
                               ncol=ncol, 
                               delr=delr, 
                               delc=delc, 
                               top=z[0], 
                               botm=z[1:], 
                              )

# aquifer properties
gwf_npf  = fp.mf6.ModflowGwfnpf(model=gwf, 
                                k=k, # horizontal k value
                                save_flows=True, # save the flow for all cells
                               )
    
# initial condition
gwf_ic = fp.mf6.ModflowGwfic(model=gwf, 
                             strt=hR, # initial head used for iterative solution
                            )

# wells
wellin = [[(0, 0, 0), U, Tinj]] # [(layer, row, col), U, temperature] during injection
wel_spd = {0: wellin} # stress period data for period 0
gwf_wel = fp.mf6.ModflowGwfwel(model=gwf, 
                               stress_period_data=wel_spd, 
                               auxiliary=['TEMPERATURE'], # the last entry is specified as temperature
                               pname='WEL1', # package name
                              )

# constant head 
chd0 = [[(0, 0, ncol - 1), hR, Tinit]] # [(layer, row, col), head, temperature]
chd_spd  = {0: chd0} # stress period data
gwf_chd = fp.mf6.ModflowGwfchd(model=gwf, 
                               stress_period_data=chd_spd, 
                               auxiliary=['TEMPERATURE'], # the last entry is specified as temperature
                               pname='CHD1', # package name
                              )
    
# output control
gwf_oc = fp.mf6.ModflowGwfoc(model=gwf, 
                             saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")], # what to save
                             budget_filerecord=f"{gwfname}.cbc", # file name where all budget output is stored
                             head_filerecord=f"{gwfname}.hds", # file name where all head output is stored
                            )

## Create groundwater energy model (`gwe`)
A groundwater energy model is added to the simulation. The energy model is stored in the variable `gwe`.
The following features are added to the groundwater energy model `gwe`:
* Iterative model solver \
  A separate iterative solver instance must be defined and added to the simulation `sim`, even when all solver settings are the same as for the groundwater flow model. The iterative model solver must include a file name that is different from the file name used for the groundwater flow model. As for the `gwf` model, the iterative model solver must be registered with the simulation.
* Spatial discretization \
  All function arguments are the same as for the groundwater flow model. It may seem silly to have to define the discretization again, but it is actually possible to do an energy model without a groundwater flow model (but we are not doing that).
* Initial conditions \
  Specify the initial temperature. Here it is constant, so one value is sufficient, but this may also be an array with a different value for each cell. 
* Storage and transfer \
  Specify the porosity of each cell (which is constant here but may be different for each cell), and the heat capacity and density of both the water and the solids.
* Advection \
  Specify what solution method to use for the advection term. We use the (second order implicit) total variation diminishing (TVD) option, which is currently the most accurate method in MF6 (even more accurate methods are under development).
* Conduction \
  Specify the dispersivity (both longitudinal and transversal, even though transversal is not used in this model yet) and the thermal conductivity of both the water and the solids. The conduction package plays a similar role as the dispersion package in solute transport modeling. 
* Source sink mixing \
  Specify how temperature is assigned for sources and sinks in the flow model. We have both wells and constant head cells for which we specify the temperature through a `'TEMPERATURE'` auxiliary variable. Use the `pname` strings that are specified for the WEL and CHD packages.
* Output control \
  Similar to the output control of the groundwater flow model, but now save the temperature rather than the head.

In [ ]:
# groundwater energy model
gwe = fp.mf6.ModflowGwe(simulation=sim, # add to simulation called sim
                        modelname=gwename, # name of gwe model
                        save_flows=True, # make sure all flows are stored in binary output file
                       )

# iterative model solver
gwe_ims  = fp.mf6.ModflowIms(simulation=sim, # add to simulation
                             filename=gwe.name + '.ims', # must be different than file name of gwf model ims
                             linear_acceleration="BICGSTAB",
                             outer_dvclose=1e-6,
                            ) 

# register solver
sim.register_ims_package(solution_file=gwe_ims, 
                         model_list=[gwe.name],
                        )

# discretization
gwe_dis = fp.mf6.ModflowGwedis(model=gwe, # add to gwe model
                               nlay=nlay, 
                               nrow=nrow, 
                               ncol=ncol, 
                               delr=delr, 
                               delc=delc, 
                               top=z[0], 
                               botm=z[1:], 
                              )

# initial condition
gwe_ic = fp.mf6.ModflowGweic(model=gwe, 
                             strt=Tinit, # initial temperature
                            ) 

# energy storage package
gwe_est = fp.mf6.ModflowGweest(model=gwe, 
                               porosity=theta, # porosity
                               heat_capacity_water=cpw,
                               density_water=rhow,
                               heat_capacity_solid=cps,
                               density_solid=rhos,
                               pname="EST",
                               save_flows=True,
                              )

# advection
gwe_adv = fp.mf6.ModflowGweadv(model=gwe,  
                           scheme="TVD", # use the tvd method
                           pname='ADV1',
                          )

# conduction
gwe_cnd = fp.mf6.ModflowGwecnd(model=gwe, 
                           xt3d_off=False,
                           alh=alphaL, # longitudinal dispersivity
                           ath1=alphaT, # transverse dispersivity
                           ktw=kappaw, # thermal conductivity of water
                           kts=kappas, # thermal conductivity of solids
                           pname='CND', 
                          )

# source sink mixing
sourcelist = [("WEL1", "AUX", "TEMPERATURE"), ("CHD1", "AUX", "TEMPERATURE")] # list of (pname, 'AUX', 'TEMPERATURE')
gwe_ssm = fp.mf6.ModflowGwessm(model=gwe, 
                           sources=sourcelist, 
                           save_flows=True,
                           pname='SSM1', 
                          )

# output control
gwe_oc = fp.mf6.ModflowGweoc(model=gwe,
                         saverecord=[("TEMPERATURE", "ALL"), ("BUDGET", "ALL")], # what to save
                         budget_filerecord=f"{gwename}.cbc", # file name where all budget output is stored
                         temperature_filerecord=f"{gwename}.ucn", # file name where all temperature output is stored
                        )

Define the interaction between the groundwater flow model and the energy model and add it to the simulation.

In [ ]:
gwf_gwe = fp.mf6.ModflowGwfgwe(simulation=sim, 
                               exgtype="GWF6-GWE6", 
                               exgmnamea=gwf.name, # name of groundwater flow model 
                               exgmnameb=gwe.name, # name of groundwater energy model
                               filename=f"{modelname}.gwfgwe",
                              )

## Write input files and solve model
We are finally ready to write the input files and solve the model. First, we write the input files. This will by default print a lot of information to the screen (but we silenced it by specifying the `silent=True` option).
Note that `flopy` creates a subdirectory called `gwe1d` (the `modelws` defined in one of the first code cells of this notebook) that contains all the MODFLOW input files. Check the `modelws` directory to see what files were created. You may also inspect the contents of some of the files. Their exact format is specified in the USGS manual. FloPy takes care of all the painstaking work that is required to create these input files!

Finally, we solve the model. If all goes well, the final statement on the screen is `Normal termination of simulation.`

In [ ]:
sim.write_simulation(silent=True)
success, _ = sim.run_simulation(silent=True) 
if success == 1:
    print('Model solved successfully')
else:
    print('Solve failed')

## Read head data and make plot
The head is steady (storage is not simulated for this confined aquifer, where flows adjust very rapidly), so it should be the same for all times. 
Flow is 1D, so we should get a straight line. Head is equal to $h_R=0$ in the last cell on the right.

In [ ]:
hds = gwf.output.head() # get handle to binary head file
head = hds.get_alldata().squeeze() # get all the head data from the file
times = np.array(hds.get_times()) # get times and make it an array
print(f'min, max head in model: {head.min():.2f}, {head[0].max():.2f} m')
print(f'shape of head array (ntimes, nlay, ncol): {head.shape}')

In [ ]:
plt.plot(xg, head[0])
plt.xlabel('x (m)')
plt.ylabel('head (m)')
plt.xticks(np.linspace(0, 150, 7))
plt.grid()

## Read temperature data and make plot
The tempeature data is read and a plot of the temperature in the aquifer is created below. An `ipywidget` slider is added to show the temperature for different times.

In [ ]:
tempobj = gwe.output.temperature() # get handle to binary temperature file
temp = tempobj.get_alldata().squeeze() # get the temperature data from the file
times = np.array(tempobj.get_times()) # get the times and convert to array

In [ ]:
def plot(t):
    plt.subplot(111, xlim=(0, 150), ylim=(Tinit - 0.2, Tinj + 0.2), xlabel='x (m)', ylabel='Temperature (\u00b0C)')
    tindex = int(t / delt) - 1
    plt.plot(xg, temp[tindex])
    plt.yticks(np.linspace(Tinit, Tinj, 5))
    plt.grid()

interact(plot, t=FloatSlider(value=delt, min=delt, max=tin, 
                                description='time (days):', readout_format='.1f',
                                step=delt, layout=Layout(width='80%')));

### Compare to exact solution
The exact solution for the stated 1D problem is (modified from Van Genuchten and Alves, 1982):
\begin{equation}
T(x, t) = T_\text{init} + (T_\text{inj}-T_\text{init})\left[\frac{1}{2}\text{erfc}\left(\frac{Rx-vt}{2\sqrt{DRt}}\right) + 
\sqrt{\frac{v^2t}{\pi DR}} \exp\left(\frac{-(Rx-vt)^2}{4DRt}\right) -
\frac{1}{2}\left(1 + \frac{vx}{D} + \frac{v^2t}{DR}\right)\exp\left(\frac{vx}{D}\right)
\text{erfc}\left(\frac{Rx+vt}{2\sqrt{DRt}}\right)\right]
\end{equation}
where the thermal dispersion coefficient $D$ is defined as
\begin{equation}
D = \frac{\theta \kappa_w + (1 - \theta) \kappa_s}{\theta c_b\rho_b} + \alpha_L v_\text{avg}
\end{equation}
with the bulk specific heat capacity (volume based) defined  as
\begin{equation}
c_b\rho_b = \theta c_w \rho_w + (1 - \theta) c_s \rho_s
\end{equation}
where $\theta$ is the porosity, $\kappa_w$ and $\kappa_s$ are the thermal conductivity of the water and solids, $c_w$ and $c_s$ are the specific heat capacity (mass based) of water and solids, $\rho_w$ and $\rho_s$ are the densies of water and solids, $\alpha_L$ is the longitudianal dispersivity, and $v_\text{avg}$ is the average velocity. The thermal retardation coefficient $R$ is defined as 
\begin{equation}
R = \frac{c_b\rho_b}{\theta c_w \rho_w}
\end{equation}
Finally, $\text{erfc}$ is the complimentary error function. The exact solution is plotted in the same graph as the MODFLOW solution. The MODFLOW solution is quite accurate and shows only a bit of numerical dispersion. 

In [ ]:
from scipy.special import erfc

def T_exact_novec(x, t, v, R, D, Tinit, Tinj):
    rv1 = 0.5 * erfc((R * x - v * t) / (2 * np.sqrt(D * R * t)))
    rv2 = np.sqrt(v ** 2 * t / (np.pi * D * R)) * np.exp(-(R * x - v * t) ** 2 / (4 * D  * R * t))
    arg1 = v * x / D
    arg2 = (R * x + v * t) / (2 * np.sqrt(D * R * t))
    if arg1 > 20: # use first terms of series expansion
        rv3 = 0.5 * (1 + v * x / D + v ** 2 * t / (D * R)) * np.exp(arg1 - arg2 ** 2) / (
              np.sqrt(np.pi) * arg2)
    else:
        rv3 = 0.5 * (1 + v * x / D + v ** 2 * t / (D * R)) * np.exp(arg1) * \
              erfc(arg2)    
    rv = rv1 + rv2 - rv3
    rv = np.nan_to_num(rv)
    rv = Tinit + (Tinj - Tinit) * rv
    return rv

T_exact = np.vectorize(T_exact_novec)

In [ ]:
cbrhob = theta * cpw * rhow + (1 - theta) * cps * rhos
R = cbrhob / (theta * cpw * rhow)
D = (theta * kappaw + (1 - theta) * kappas) / (theta * cbrhob) + alphaL * vavg
print(f'Thermal retardation coefficient: {R:.3f}')
print(f'Thermal diffusion coefficient: {D:.3f} m^2/d')

In [ ]:
for t in np.arange(30, 181, 30):
    itime = int(t / delt) - 1 # find index of value in times closest to t
    plt.plot(xg, temp[itime], label=f't={times[itime]:.0f} d')
    Tex = T_exact(xg, t, vavg, R, D, Tinit, Tinj)
    plt.plot(xg, Tex, 'k--')
    plt.yticks([12, 14.5, 17])
plt.xlabel('x (m)')
plt.ylabel('temperature (\u2103)') 
plt.title('colored: modflow, black dashes: exact')
plt.legend()
plt.grid()

### Energy balance
The energy balance is computed by taking the difference between the total energy in the system at the end of the simulation and at the beginning of the simulation and comparing that to the total inflow minus the total outflow of energy. The energy in the system at the beginning of the simulation is
\begin{equation}
E_\text{begin} = c_b\rho_bT_\text{init}LH
\end{equation}
where $T_\text{init}$ is the initial temperature, $c_b\rho_b$ is the bulk specific heat capacity, $L$ is the length of the domain, and $H$ is the aquifer thickness.
The energy in the system at the end of the simulation is 
\begin{equation}
E_\text{end} = c_b\rho_b\sum T_i\Delta_x H
\end{equation}
where $T_i$ is the temperature in cell $i$ and $\Delta_x$ is the length of a cell (`delr` in MODFLOW).

The energy inflow during injection is
\begin{equation}
E_\text{inflow} = UT_\text{inj}c_b\rho_b t_\text{in}
\end{equation}
while the energy outlfow on the right side of the model is
\begin{equation}
E_\text{outflow} = UT_\text{init} c_b\rho_b t_\text{in}
\end{equation}
where it is assumed that the model domain is large enough such that the temperature in the last cell remains equal to $T_\text{init}$.

Note that the energy balance is met exactly (within machine accuracy) even though the MODFLOW solution has some numerical dispersion. 

In [ ]:
cbrhob = theta * cpw * rhow + (1 - theta) * cps * rhos
energy_begin = cbrhob * Tinit * L * H
energy_end = cbrhob * np.sum(temp[-1] * delr * H)

print(f'total change of energy: {energy_end - energy_begin:.9e} J')

In [ ]:
energy_inflow = U * Tinj * cpw * rhow * tin # left side of model
energy_outflow = U * Tinit * cpw * rhow * tin # right side of model
print(f'total inflow of energy: {energy_inflow - energy_outflow:.9e} J')

### Exercise 1 Reduce numerical dispersion

Let's see if the solution gets more accurate (the numerical dispersion decreases) when we reduce the time step. What time step gives an accurate solution?

## Recovery phase
After injection, the water is recovered again by pumping the water out. The extraction rate is chosen the same as the injection rate. 
The model now has two stress periods: injection during the first stress period and extraction during the second stress period. The following parameters are modified or added:

In [ ]:
# time and time discretization
tin = 180 # injection time, d
delt = 1 # time step, d
nstepin = round(tin / delt) # computed number of steps during injection, integer
tout = 180 # extraction time, d
delt = 1 # time step, d
nstepout = round(tout / delt) # computed number of steps during extraction, integer

The entire model (simulation, groundwater flow model, groundwater transport model, and interaction) are gathered in one function. Inside the function, the model is built and solved. The model takes no input arguments; all parameters are taken from the parameter blocks defined above. The function returns the computed temperature and corresponding times. Only two modifications need to be made to the simulation and groundwater flow model developed above; no changes need to be made to the groundwater transport model. The changes are:
* The time discretization of the simulation is modified to reflect two stress periods.
* The well package of the groundwater flow model is modified to simulate the injection and extraction rates as specified for the two stress periods.

In [ ]:
def atesmodel(ncycle=1):
    # simulation
    sim = fp.mf6.MFSimulation(sim_name=modelname, # name of simulation
                              version='mf6', # version of MODFLOW
                              exe_name='mf6', # path to MODFLOW executable
                              sim_ws=modelws, # path to workspace where all files are stored
                             )
    
    # time discretization
    tdis = fp.mf6.ModflowTdis(simulation=sim, # add to the simulation called sim (defined above)
                              time_units="DAYS", 
                              nper=2 * ncycle, # number of stress periods 
                              perioddata=ncycle * [[tin, nstepin, 1], # period length, number of steps, timestep multiplier
                                        [tout, nstepout, 1]],
                             )

    # groundwater flow model
    gwf = fp.mf6.ModflowGwf(simulation=sim, # add to simulation called sim
                            modelname=gwfname, # name of gwf model
                            save_flows=True, # make sure all flows are stored in binary output file
                           )
    
    # iterative model solver
    gwf_ims  = fp.mf6.ModflowIms(simulation=sim, # add to simulation called sim
                                 filename=gwf.name + '.ims', # file name to store ims
                                 linear_acceleration="BICGSTAB", # use BIConjuGantGradientSTABalized method
                                )                                                                                                
    # register solver
    sim.register_ims_package(solution_file=gwf_ims, # name of iterative model solver instance
                             model_list=[gwf.name], # list with name of groundwater flow model
                            )   
    
    # discretization
    gwf_dis = fp.mf6.ModflowGwfdis(model=gwf, # add to groundwater flow model called gwf
                                   nlay=nlay, 
                                   nrow=nrow, 
                                   ncol=ncol, 
                                   delr=delr, 
                                   delc=delc, 
                                   top=z[0], 
                                   botm=z[1:], 
                                  )
    
    # aquifer properties
    gwf_npf  = fp.mf6.ModflowGwfnpf(model=gwf, 
                                    k=k, # horizontal k value
                                    save_flows=True, # save the flow for all cells
                                   )
        
    # initial condition
    gwf_ic = fp.mf6.ModflowGwfic(model=gwf, 
                                 strt=hR, # initial head used for iterative solution
                                )
    
    # wells
    wellin = [[(0, 0, 0), U, Tinj]] # [(layer, row, col), U, temperature] during injection
    wellout = [[(0, 0, 0), -U, Tinj]] # [(layer, row, col), U, temperature] during injection
    wel_spd = {}
    for i in range(ncycle):
        wel_spd[2 * i] = wellin
        wel_spd[2 * i + 1] = wellout
    #wel_spd = {0: wellin, 1:wellout} # stress period data for periods 0 and 1
    gwf_wel = fp.mf6.ModflowGwfwel(model=gwf, 
                                   stress_period_data=wel_spd, 
                                   auxiliary=['TEMPERATURE'],
                                   pname='WEL1', # package name
                                  )
    
    # constant head 
    chd0 = [[(0, 0, ncol - 1), hR, Tinit]] # [(layer, row, col), head, temperature]
    chd_spd  = {0: chd0} # stress period data
    gwf_chd = fp.mf6.ModflowGwfchd(model=gwf, 
                                   stress_period_data=chd_spd, 
                                   auxiliary=['TEMPERATURE'],
                                   pname='CHD1', # package name
                                  )
        
    # output control
    gwf_oc = fp.mf6.ModflowGwfoc(model=gwf, 
                                 saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")], # what to save
                                 budget_filerecord=f"{gwfname}.cbc", # file name where all budget output is stored
                                 head_filerecord=f"{gwfname}.hds", # file name where all head output is stored
                                )

    # groundwater energy model
    gwe = fp.mf6.ModflowGwe(simulation=sim, # add to simulation called sim
                            modelname=gwename, # name of gwe model
                            save_flows=True, # make sure all flows are stored in binary output file
                           )
    
    # iterative model solver
    gwe_ims  = fp.mf6.ModflowIms(simulation=sim, # add to simulation
                                 filename=gwe.name + '.ims', # must be different than file name of gwf model ims
                                 linear_acceleration="BICGSTAB",
                                 outer_dvclose=1e-6,
                                ) 
    
    # register solver
    sim.register_ims_package(solution_file=gwe_ims, 
                             model_list=[gwe.name],
                            )
    
    # discretization
    gwe_dis = fp.mf6.ModflowGwedis(model=gwe, # add to gwe model
                                   nlay=nlay, 
                                   nrow=nrow, 
                                   ncol=ncol, 
                                   delr=delr, 
                                   delc=delc, 
                                   top=z[0], 
                                   botm=z[1:], 
                                  )
    
    # initial condition
    gwe_ic = fp.mf6.ModflowGweic(model=gwe, 
                                 strt=Tinit, # initial temperature
                                ) 
    
    # energy storage package
    gwe_est = fp.mf6.ModflowGweest(model=gwe, 
                                   porosity=theta, # porosity
                                   heat_capacity_water=cpw,
                                   density_water=rhow,
                                   heat_capacity_solid=cps,
                                   density_solid=rhos,
                                   pname="EST",
                                   save_flows=True,
                                  )
    
    # advection
    gwe_adv = fp.mf6.ModflowGweadv(model=gwe,  
                               scheme="TVD", # use the tvd method
                               pname='ADV1',
                              )
    
    # conduction
    gwe_cnd = fp.mf6.ModflowGwecnd(model=gwe, 
                               xt3d_off=False,
                               alh=alphaL, # longitudinal dispersivity
                               ath1=alphaT, # transverse dispersivity
                               ktw=kappaw, # thermal conductivity of water
                               kts=kappas, # thermal conductivity of solids
                               pname='CND', 
                              )
    
    # source sink mixing
    sourcelist = [("WEL1", "AUX", "TEMPERATURE"), ("CHD1", "AUX", "TEMPERATURE")] # list of (pname, 'AUX', 'TEMPERATURE')
    gwe_ssm = fp.mf6.ModflowGwessm(model=gwe, 
                               sources=sourcelist, 
                               save_flows=True,
                               pname='SSM1', 
                              )
    
    # output control
    gwe_oc = fp.mf6.ModflowGweoc(model=gwe,
                             saverecord=[("TEMPERATURE", "ALL"), ("BUDGET", "ALL")], # what to save
                             budget_filerecord=f"{gwename}.cbc", # file name where all budget output is stored
                             temperature_filerecord=f"{gwename}.ucn", # file name where all temperature output is stored
                            )

    gwf_gwe = fp.mf6.ModflowGwfgwe(simulation=sim, 
                                   exgtype="GWF6-GWE6", 
                                   exgmnamea=gwf.name, # name of groundwater flow model 
                                   exgmnameb=gwe.name, # name of groundwater energy model
                                   filename=f"{modelname}.gwfgwe",
                                  )

    sim.write_simulation(silent=True)
    success, _ = sim.run_simulation(silent=True) 
    if success == 1:
        print('Model solved successfully')
    else:
        print('Solve failed')

    # read temperature output
    tempobj = gwe.output.temperature() # get handle to binary temperature file
    temp = tempobj.get_alldata().squeeze() # get the temperature data from the file
    times = np.array(tempobj.get_times()) # get the times and convert to array

    return temp, times
        

In [ ]:
# run model
ncycle = 3
temp, times = atesmodel(ncycle)

In [ ]:
from ipywidgets import interact

def plot(t):
    tindex = int(t / delt) - 1
    plt.subplot(111, xlim=(0, L), ylim=(Tinit - 0.2, Tinj + 0.2), xlabel='x (m)', ylabel='Temperature (\u00b0C)')
    plt.plot(xg, temp[tindex])
    plt.grid()

interact(plot, t=FloatSlider(value=delt, min=delt, max=ncycle * (tin + tout), 
                             description='time (days):', readout_format='.1f',
                             step=delt, layout=Layout(width='80%')));

## Thermal recovery efficiency
The temperature in the first cell is plotted vs. time in the code cell below. Recall that water is extracted between $t=180$ and $t=360$, then again between $t=540$ and $t=720$, and between $t=900$ and $t=1080$ d. Note that the temperature of the extracted water at the end of the extraction period doesn't go down as much for later cycles. 

In [ ]:
plt.plot(times, temp[:, 0])
plt.xlabel('time (d)')
plt.ylabel('temperature at $x=0$ (\u2103)')
plt.xticks(np.linspace(0, ncycle * 360, 2 * ncycle + 1))
plt.ylim(Tinit - 0.2, Tinj + 0.2)
plt.grid()

The thermal recovery efficiency $R_\text{eff}$ of a cycle is computed as the total recovered energy during a cycle
\begin{equation}
E_\text{inj} = \frac{U(T_\text{inj} - T_\text{init})c_w\rho_w t_\text{in}}{
\sum_n{U (T_n - T_\text{init})c_w\rho_w \Delta t}}
\end{equation}
where $T_n$ is the temperature in cell 0 during time step $n$.

In [ ]:
for i in range(ncycle):
    energy_injected = U * (Tinj - Tinit) * cpw * rhow * tin
    energy_extracted = np.sum(U * (temp[nstepin + i * (nstepin + nstepout): (i + 1) * (nstepin + nstepout), 0] - Tinit) * cpw * rhow * delt)
    print(f'total energy injected:  {energy_injected:.8e} J')
    print(f'total energy extracted: {energy_extracted:.8e} J')
    print(f'thermal recovery efficiency: {energy_extracted * 100 / energy_injected:.1f}%')

### Exercise 2
Investigate the effect of the longitudinal dispersivity on the thermal recovery efficiency. 
What is the thermal recovery efficiency of the first three cycles when $\alpha_L=0.5$ m?

### Hint
Modify the `atesmodel` function so that it takes `alphaL` as an input argument. Then run the modified function with `alphaL=0.5` and recompute the recovery efficiency with the code from the code cell above. 

### Reference
Van Genuchten, M.T., and W.J. Alves. 1982. Analytical solutions of the one-dimensional convective-dispersive solute transport equation (No. 1661). US Department of Agriculture, Agricultural Research Service.